In [1]:
print("ok")

ok


In [2]:
%pwd

'f:\\Data Science\\Projects\\Gen AI\\Medical-Chatbot-Generative-AI\\research'

In [85]:
import os

In [86]:
os.chdir("../")

In [87]:
%pwd

'f:\\Data Science\\Projects\\Gen AI'

In [88]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [45]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [49]:
extracted_data=load_pdf_file(data='Data/')

In [50]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [89]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 7020


In [90]:
len(text_chunks)

7020

In [91]:
from langchain.embeddings import HuggingFaceEmbeddings

In [92]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [93]:
embeddings = download_hugging_face_embeddings()

In [94]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [100]:
from dotenv import load_dotenv
load_dotenv()

False

In [104]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Check if the API key is loaded correctly
pinecone_api_key = os.getenv("PINECONE_API_KEY")
if pinecone_api_key is None:
    raise ValueError("Pinecone API key is missing. Please check your .env file.")

print(f"Pinecone API Key: {pinecone_api_key}")  # For debugging purposes, remove this after checking



Pinecone API Key: db1a1112-762b-43a0-9832-704a6eba3213


In [105]:
from dotenv import load_dotenv
load_dotenv()

False

In [106]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [41]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [108]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [54]:
# Embed each chunk and insert the embeddings into your Pinecone Index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [109]:
# Load Existing index 
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [110]:
docsearch

In [111]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [112]:
retrieved_docs = retriever.invoke("What is Acne?")

In [113]:
retrieved_docs

[Document(id='77667c72-efe6-489d-94a7-ea402f7e116d', metadata={'page': 37.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Acidosis seeRespiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is'),
 Document(id='2f1f090e-7695-4985-81d3-51ec901cdc7b', metadata={'page': 38.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='3424c767-0a63-4b29-83c9-9897dc448f78', metadata={'page': 239.0, 'sou

In [119]:
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline

# Load a Hugging Face model for text generation (e.g., GPT-2)
generator = pipeline("text-generation", model="gpt2", max_length=500, temperature=0.4, truncation=True, pad_token_id=50256)

# Wrap the Hugging Face pipeline in LangChain
llm = HuggingFacePipeline(pipeline=generator)


In [120]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [121]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [123]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

System: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.

Acidosis seeRespiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin disease characterized by
pimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.
Description
Acne vulgaris, the medical term for common acne, is

GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne
Acne vulgaris affecting a woman’s face. Acne is the general
name given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-
ates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25

ent purposes. For example, lotions, soaps, gels, andcreams containing benzoyl peroxide or tretinoin may beused